In [1]:
#from sklearn import preprocessing
import numpy as np     
import pandas as pd   
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

In [2]:
cd 

C:\Users\JDL


In [3]:
cd Desktop

C:\Users\JDL\Desktop


In [4]:
power= pd.read_csv('2019 태양광 발전 경진대회/Refined Data/please.csv', index_col=0)

In [5]:
power.set_index(power['일시'], inplace=True)
power['일시'] = pd.to_datetime(power['일시'], errors='coerce')
power['시간'] = power['hour']
power['월'] = power['month']
power = power.drop(['일시', 'hour', 'month'], axis=1)

In [6]:
split_date = '2019-04-01'

power_train = power.loc[power.index < split_date]
power_test = power.loc[power.index >= split_date]

x_train_final = power_train[[ '일조(hr)', '일사(MJ/m2)', '지면온도(°C)', '시간']]
x_test_final = power_test[['일조(hr)', '일사(MJ/m2)', '지면온도(°C)', '시간']]
y_train = power_train['태양광 전력거래량\n(단위: MWh)']
y_test = power_test['태양광 전력거래량\n(단위: MWh)']    

In [7]:
n = np.arange(200, 1001, 25)

In [8]:
cv = 5
grid = {
        'n_estimator': n
        }

model = xgb.XGBRegressor()
gridsearch = GridSearchCV(model, grid, verbose=2 , n_jobs=5,
         cv=TimeSeriesSplit(n_splits=cv).get_n_splits([x_train_final, y_train]))
gridsearch.fit(x_train_final, y_train)

df_gridsearch = pd.DataFrame(gridsearch.cv_results_)

Fitting 5 folds for each of 33 candidates, totalling 165 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    8.3s
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed:   19.7s
[Parallel(n_jobs=5)]: Done 165 out of 165 | elapsed:   21.4s finished
C:\Users\JDL\Anaconda3\envs\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\JDL\Anaconda3\envs\venv\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\JDL\Anaconda3\envs\venv\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kw

In [10]:
gridsearch.best_params_

{'n_estimator': 200}

In [11]:
cv = 5
grid = {
        'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'min_child_weight': [1, 2, 3, 4, 5, 6, 7, 8, 9],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
        }


model = xgb.XGBRegressor(n_estimator=200)
gridsearch = GridSearchCV(model, grid, verbose=2 , n_jobs=5,
         cv=TimeSeriesSplit(n_splits=cv).get_n_splits([x_train_final, y_train]))
gridsearch.fit(x_train_final, y_train)


df_gridsearch = pd.DataFrame(gridsearch.cv_results_)
df_gridsearch.to_csv('solar_grid.csv')

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    3.3s
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed:   15.9s
[Parallel(n_jobs=5)]: Done 355 tasks      | elapsed:   37.5s
[Parallel(n_jobs=5)]: Done 638 tasks      | elapsed:  1.2min
[Parallel(n_jobs=5)]: Done 1003 tasks      | elapsed:  2.4min
[Parallel(n_jobs=5)]: Done 1448 tasks      | elapsed:  4.6min
[Parallel(n_jobs=5)]: Done 1975 tasks      | elapsed:  6.4min
[Parallel(n_jobs=5)]: Done 2582 tasks      | elapsed:  9.3min
[Parallel(n_jobs=5)]: Done 2700 out of 2700 | elapsed:  9.9min finished


In [12]:
gridsearch.best_params_

{'max_depth': 4, 'min_child_weight': 8, 'subsample': 0.7}